In [2]:
!pip install biopython googletrans==4.0.0-rc1 > /dev/null 2>&1

from Bio import Entrez
import pandas as pd
from googletrans import Translator

# Configuration de l'API PubMed
Entrez.email = "worthylastchance@gmail.com"

translator = Translator()

# Fonction pour rechercher des articles PubMed en fonction d'une requête
def search_pubmed(query, max_results=5):
    handle = Entrez.esearch(db="pubmed", term=query, retmax=max_results)
    record = Entrez.read(handle)
    handle.close()
    return record["IdList"]

def fetch_article_details(pubmed_id):
    handle = Entrez.efetch(db="pubmed", id=pubmed_id, rettype="xml")
    record = Entrez.read(handle)
    handle.close()

    article = record['PubmedArticle'][0]['MedlineCitation']['Article']
    title = article['ArticleTitle']
    abstract = article['Abstract']['AbstractText'][0] if 'Abstract' in article else "Résumé non disponible"
    journal = article['Journal']['Title']
    pub_date = article['Journal']['JournalIssue']['PubDate'].get('Year', 'Date inconnue')
    doi = article.get('ELocationID', ['DOI non disponible'])[0]
    authors = ", ".join([f"{author['ForeName']} {author['LastName']}"
                         for author in article.get("AuthorList", [])]) if "AuthorList" in article else "Auteurs non disponibles"

    # Extraction correcte des mots-clés
    keyword_list = record['PubmedArticle'][0]['MedlineCitation'].get('KeywordList', [])
    keywords = ", ".join([str(keyword) for keyword in keyword_list])

    return {
        "ID": pubmed_id,
        "Title": title,
        "Abstract": abstract,
        "Journal": journal,
        "PublicationDate": pub_date,
        "DOI": doi,
        "Authors": authors,
        "Keywords": keywords
    }

    return {
        "ID": pubmed_id,
        "Title": title,
        "Abstract": abstract,
        "Journal": journal,
        "PublicationDate": pub_date,
        "DOI": doi,
        "Authors": authors,
        "Keywords": keywords
    }

# Fonction pour générer un rapport d'article
def generate_report(article, language="en"):
    print("\n--- Rapport d'Article ---")
    print(f"ID : {article['ID']}")
    print(f"Titre : {article['Title']}")
    print(f"Résumé (original) : {article['Abstract']}")
    print(f"Journal : {article['Journal']}")
    print(f"Date de publication : {article['PublicationDate']}")
    print(f"Auteurs : {article['Authors']}")
    print(f"Mots-clés : {article['Keywords']}")
    print(f"DOI : {article['DOI']}")

    # Résumé traduit
    translated_summary = translator.translate(article['Abstract'], dest=language).text
    print(f"\nRésumé traduit ({language}) : {translated_summary}")
    print("--------------------------")

# Fonction principale pour le chatbot interactif
def chatbot_mode():
    print("Bienvenue dans le chatbot interactif pour PubMed.")
    saved_articles = []

    while True:
        print("\nQue voulez-vous faire ?")
        print("1. Rechercher des articles")
        print("2. Résumer un article avec un ID spécifique (dans une langue choisie)")
        print("3. Afficher les détails d'un article avec un ID spécifique")
        print("4. Quitter")

        choice = input("Entrez votre choix (1, 2, 3 ou 4) : ")

        if choice == "1":
            query = input("\nEntrez votre requête de recherche : ")
            print("\nRecherche en cours...")
            pubmed_ids = search_pubmed(query, max_results=5)

            if not pubmed_ids:
                print("Aucun article trouvé pour cette requête.")
                continue

            articles = [fetch_article_details(pubmed_id) for pubmed_id in pubmed_ids]
            saved_articles.extend(articles)

            # Sauvegarde dans un fichier CSV
            file_path = "pubmed_articles.csv"
            pd.DataFrame(saved_articles).to_csv(file_path, index=False)
            print(f"\nLes articles ont été sauvegardés dans le fichier : {file_path}")

            for article in articles:
                print("\n------------------------------------------")
                print(f"ID : {article['ID']}")
                print(f"Titre : {article['Title']}")
                print(f"Résumé : {article['Abstract']}")
                print(f"Journal : {article['Journal']}")
                print(f"Date de publication : {article['PublicationDate']}")
                print(f"DOI : {article['DOI']}")
                print("------------------------------------------")

        elif choice == "2":
            if not saved_articles:
                print("Aucun article sauvegardé. Effectuez une recherche d'abord.")
                continue

            article_id = input("Entrez l'ID de l'article à résumer : ")
            language = input("Entrez la langue souhaitée (par ex., 'fr' pour français, 'en' pour anglais) : ").strip().lower()
            article = next((art for art in saved_articles if art["ID"] == article_id), None)
            if article:
                generate_report(article, language)
            else:
                print(f"Aucun article trouvé avec l'ID {article_id}.")

        elif choice == "3":
            if not saved_articles:
                print("Aucun article sauvegardé. Effectuez une recherche d'abord.")
                continue

            article_id = input("Entrez l'ID de l'article pour les détails : ")
            article = next((art for art in saved_articles if art["ID"] == article_id), None)
            if article:
                print("\n--- Détails supplémentaires de l'article ---")
                print(f"Titre : {article['Title']}")
                print(f"Auteurs : {article['Authors']}")
                print(f"Mots-clés : {article['Keywords']}")
                print(f"Résumé : {article['Abstract']}")
                print(f"Journal : {article['Journal']}")
                print(f"DOI : {article['DOI']}")
                print("------------------------------------------")
            else:
                print(f"Aucun article trouvé avec l'ID {article_id}.")

        elif choice == "4":
            print("Merci d'avoir utilisé le chatbot PubMed. À bientôt !")
            break

        else:
            print("Choix invalide. Veuillez réessayer.")

# Lancement du mode chatbot
if __name__ == "__main__":
    chatbot_mode()


Bienvenue dans le chatbot interactif pour PubMed.

Que voulez-vous faire ?
1. Rechercher des articles
2. Résumer un article avec un ID spécifique (dans une langue choisie)
3. Afficher les détails d'un article avec un ID spécifique
4. Quitter
Entrez votre choix (1, 2, 3 ou 4) : 1

Entrez votre requête de recherche : Breast Cancer Biomarkers

Recherche en cours...

Les articles ont été sauvegardés dans le fichier : pubmed_articles.csv

------------------------------------------
ID : 39710525
Titre : Breast Cancers With Intermediate Estrogen Receptor Expression: Characteristics, Prognosis and Treatment.
Résumé : In the era of personalized oncology biomarkers that identify subgroups of specific cancers and help predict response to specific therapies are critical tools for prognosis determination and therapeutic decisions. The Estrogen Receptor (ER) had been one of the first biomarkers used in breast cancer and has helped advance the field of breast oncology by contributing to the success o